In [49]:
import math
import json
import datetime as time

In [53]:
directory = 'D:\Games\steamapps\common\Kerbal Space Program\saves\Ship Testing\Ships\VAB\\'
fileExt = ".craft"

fileName = "ElectTest1"

In [201]:
def parseData(rawData) :
    data = []
    lines = rawData.split("\n")
    for d in lines :
        splitData = d.strip("\t").split(" = ")
        for spl in splitData :
            spl.strip()
        if len(splitData) > 1 :
            data.append(splitData)
    return data

def getPartNames(data) :
    output = []
    for d in data :
        if d[0] == "part" :
            d[1] = d[1].split("_")[0]
            output.append(d)
    return output

In [202]:
with open(directory + fileName + fileExt) as file :
    data = []
    rawData = file.read()
    data = parseData(rawData)
    partNames = getPartNames(data)

parts = []
for p in partNames :
    parts.append(p[1])
    

for p in parts :
    print(p)

HECS2.ProbeCore
trussPiece1x
SCANsat.Scanner24
batteryBank
advSasModule
fuelTankSmall
liquidEngineMini.v2
solarPanels4
solarPanels4
spotLight1
spotLight1
spotLight1
spotLight1
spotLight1
spotLight1
spotLight1
spotLight1


In [203]:
### Dealing with JSON Library and loading the needed data ###

In [204]:
jsonPath = "D:\Games\steamapps\common\Kerbal Space Program\Ships\Script\PartData\\KerbalElectricalData.json"

In [205]:
def parseInt(x) :
    try :
        return int(x)
    except :
        return 11111111
    
def parseFloat(x) :
    try :
        return float(x)
    except :
        return 111111111111
    
class EData :
    name = ""
    FileName = ""
    Load = 0
    LoadScale = "sec"
    RWLoad = 0
    RwScale = "sec"
    Production = 0
    Battery = 0
    
    def __init__(self, name, FileName, Load, LoadScale, RWLoad, RWScale, Production, Battery) :
        self.name = name
        self.FileName = FileName
        self.Load = Load
        self.LoadScale = LoadScale
        self.RWLoad = RWLoad
        self.RWScale = RWScale
        self.Production = Production
        self.Battery = Battery
        
    def parseDict(d) :
        return EData(d["name"],d["FileName"],d["Load"],d["LoadScale"],d["RWLoad"],d["RWScale"],d["Production"],d["Battery"])
    
    def ToString(self) :
        return f"{self.name} \n   File Name: {self.FileName} \n   Load: {self.Load} ec/{self.LoadScale} \n   RWLoad: {self.RWLoad} ec/{self.RWScale} \n   Prod: {self.Production} ec/sec \n   Batt: {self.Battery} ec\n"

In [206]:
with open(jsonPath) as jsonFile :
    output = json.load(jsonFile, parse_int=parseInt, parse_float=parseFloat)
    
allEData = []
for i in output :
    for j in output[i] :
        allEData.append(EData.parseDict(j))

In [207]:
### Comparing the craft data to the json data and getting a true parts list ###

In [208]:
def getCalcPartsList(partsList) :
    output = []
    for p in partsList :
        for e in allEData :
            if e.FileName == p :
                output.append(e)
    return output

In [209]:
calcPartsList = []

calcPartsList = getCalcPartsList(parts)

In [210]:
### Calculator ###

In [214]:
class Calculator :
    parts = []
    
    totalLoad = 0
    totalProd = 0
    totalBatt = 0
    
    chargeRate = 0
    chargeTime_sec = 0
    chargeTime_time = None
    
    def __init__(self, parts) :
        self.clear()
        self.parts = parts
        
    def calculate(self) :
        self.sumTotals()
        self.calcChargeRate()
        self.calcChargeTime()
        self.convertTime()
    
    def sumTotals(self) :
        for p in self.parts :
            self.totalLoad += self.convertSeconds(p.Load, p.LoadScale)
            self.totalLoad += self.convertSeconds(p.RWLoad, p.RWScale)
            self.totalProd += p.Production
            self.totalBatt += p.Battery
            
    def calcChargeRate(self) :
        self.chargeRate = self.totalProd - self.totalLoad
    
    def calcChargeTime(self) :
        if self.chargeRate != 0 :
            self.chargeTime_sec = self.totalBatt / self.chargeRate
        else :
            self.chargeTime_sec = 0
    
    def convertSeconds(self, x, per) :
        if per == "min" :
            return x / 60
        elif per == "hr" :
            return x / math.pow(60, 2)
        else :
            return x
        
    def convertTime(self) :
        self.chargeTime_time = time.timedelta(seconds=self.chargeTime_sec)
        
    def ToString(self) :
        partsStr = ""
        for p in self.parts :
            partsStr += f"{p.ToString()}\n"
        return f"""Load : {round(self.totalLoad, 6)} ec/sec
Prod : {round(self.totalProd, 6)} ec/sec
Batt : {round(self.totalBatt, 6)} ec
        
Charge Rate : {round(self.chargeRate, 6)} ec/sec
Charge Secs : {round(self.chargeTime_sec, 6)} sec
Charge Time : {self.chargeTime_time}

Parts :
{partsStr}"""
    
    def clear(self) :
        self.parts.clear()
        self.totalLoad = 0
        self.totalProd = 0
        self.totalBatt = 0
        self.chargeRate = 0
        self.chargeTime_sec = 0
        self.chargeTime_time = None

In [215]:
calculator = Calculator(calcPartsList)

calculator.calculate()

In [216]:
print(calculator.ToString())

Load : 2.32 ec/sec
Prod : 3.28 ec/sec
Batt : 2000 ec
        
Charge Rate : 0.96 ec/sec
Charge Secs : 2083.333333 sec
Charge Time : 0:34:43.333333

Parts :
Probodobodyne_HECS2 
   File Name: HECS2.ProbeCore 
   Load: 3 ec/min 
   RWLoad: 0.5 ec/sec 
   Prod: 0 ec/sec 
   Batt: 1000 ec

SCANSAT_Biome_Scanner 
   File Name: SCANsat.Scanner24 
   Load: 1 ec/sec 
   RWLoad: 0 ec/sec 
   Prod: 0 ec/sec 
   Batt: 0 ec

Z-1K 
   File Name: batteryBank 
   Load: 0 ec/sec 
   RWLoad: 0 ec/sec 
   Prod: 0 ec/sec 
   Batt: 1000 ec

Medium_RW 
   File Name: advSasModule 
   Load: 0 ec/sec 
   RWLoad: 0.45 ec/sec 
   Prod: 0 ec/sec 
   Batt: 0 ec

OX-4L_1x6_Photovoltaic_Panels 
   File Name: solarPanels4 
   Load: 0 ec/sec 
   RWLoad: 0 ec/sec 
   Prod: 1.64 ec/sec 
   Batt: 0 ec

OX-4L_1x6_Photovoltaic_Panels 
   File Name: solarPanels4 
   Load: 0 ec/sec 
   RWLoad: 0 ec/sec 
   Prod: 1.64 ec/sec 
   Batt: 0 ec

Mk1 
   File Name: spotLight1 
   Load: 2.4 ec/min 
   RWLoad: 0 ec/sec 
   Prod: 0 e